<a href="https://colab.research.google.com/github/TheJojoJoseph/M24DE3041_MLBD_assignment4/blob/main/M24DE3041_assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Assignment 4 (M24DE4041)

implement clustering, a web search engine, and PageRank. Key files include spambase.data, webpages/, and small.txt, which must be correctly placed. The environment assumes a working local Spark setup. Libraries such as pyspark, numpy, and standard Python utilities (random, re, os, time) are used for processing and analysis.




# Part 1: Clustering

Part 1: Clustering
40 Marks
For this part of the algorithm, you have to study two new algorithms called Farthest First
algorithm for k center clustering and k-means++.
Reading for Farthest First Algorithm:

Papers and reading for K-Means++:

There are other tons of material available online in the form of slides, YouTube videos which
you can access to get a better understanding of the two algorithms.
Dataset is included in the folder: Q1- UCI Spam clustering
For this assignment we will work on points in Euclidean space represented by vectors of
numbers. The dataset is made of 4601 points with 58-dimensions which are features
available to an email spam detection system.
In Spark, the points can be represented as instances of the class
org.apache.spark.mllib.linalg.Vector and can be manipulated through static methods
Complete the coding exercises in a Jupyter notebook.
Submit your work as a .ipynb file or PDF file.
Ensure all code is properly commented and executed before submission.
Plagiarism detected will result in zero marks
Max marks: 120

http://www.wikiwand.com/en/Farthest-first_traversal

https://theory.stanford.edu/~sergei/papers/kMeansPP-soda.pdf
http://theory.stanford.edu/~sergei/slides/BATS-Means.pdf

offered by the class org.apache.spark.mllib.linalg.Vectors (these classes are available
both for Java and Python). For example, method Vectors.dense(x) transforms an array x
of double into an instance of class Vector, while method Vectors.sqdist(a,b) computes
the squared L2-distance between two instances a and b of class Vector.
You must develop 4 functions that are Spark runnable:

Make sure that kcenter(P,k) and kmeansPP(P,k) run in time O(|P| ∗ k)
Finally, you must create a python program, which receives in input a set P of features
(provided as a text file as the above dataset), and 2 integers k, k1, with k < k1. The program
incorporates the functions developed above and does the following:

Part-2: Web-search
40 Marks
1. A function readVectorsSeq(filename) that, given in input the name (or path) filename
of a text file containing the feature points (one point per line with features separated by
comma, as in the file above) transforms it into a list of Vector.
2. A function kcenter(P,k) that receives in input a set of points P and an integer k, and
returns the set C of k centers computed by the Farthest-First Traversal algorithm. You
can represent both P and C as lists of Vector.
3. A function kmeansPP(P,k) that receives in input a set of points P and an integer k, and
returns a set C of k centers computed with the kmeans++ algorithm.
4. A function kmeansObj(P,C) that receives in input a set of points P and a set of centers
C, and returns the average squared distance of a point of P from its closest center (i.e.,
the kmeans objective function for P with centers C, divided by the number of points of P).

1. Runs kcenter(P,k) printing its running time.
2. Runs kmeansPP(P,k) to obtain a set of k centers C, and then runs kmeansOb(P,C)
printing the returned value
3. Runs kcenter(P,k1) to obtain a set of k1 centers X; then runs kmeansPP(X,k) to
extarct a set of k centers C from X, and finally runs kmeansObj(P,C) printing the
returned value. Here the idea is to test whether k1 > k centers extracted with the kcenter
primitive can provide a good coreset on which running kmeans++. Of course, the larger
k1 and the better the set of centers computed by kmeansPP(X,k) .

In [23]:
# STEP 1: Install Spark (only run once)
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark

tar: spark-3.3.1-bin-hadoop3.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [24]:
import os
import sys
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
print("------CHEEEEKKKK--- Spark successfully started!")
from pyspark.mllib.linalg import Vectors
from pyspark import SparkContext
import random
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg import Vector
from pyspark.mllib.linalg import Vectors
import numpy as np
import random


------CHEEEEKKKK--- Spark successfully started!


In [ ]:
# Transforms feature points into a list of Vector.
def readVectorsSeq(filename):
    sc = SparkContext.getOrCreate()
    lines = sc.textFile(filename)
    vectors = lines.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(',')]))
    return vectors

def squared_distance(a, b):
    return sum((ai - bi) ** 2 for ai, bi in zip(a, b))


# kcenter(P,k)


In [ ]:
#receives in input a set of points P and an integer k, and returns the set C of k centers computed by the Farthest-First Traversal algorithm.
def kcenter(P, k):
    centers = []
    data = P.collect()
    first_center = random.choice(data)
    centers.append(first_center)
    for _ in range(1, k):
        max_dist = -1
        next_center = None
        for p in data:
            min_dist = min(squared_distance(p, c) for c in centers)
            if min_dist > max_dist:
                max_dist = min_dist
                next_center = p
        centers.append(next_center)
    return centers



# kmeansPP(P,k)


In [ ]:
# receives in input a set of points P and an integer k, and returns a set C of k centers computed with the kmeans++ algorithm.
def kmeansPP(P, k):
    data = P.collect()
    centers = [random.choice(data)]
    for _ in range(1, k):
        distances = []
        for p in data:
            min_dist_sq = min(squared_distance(p, c) for c in centers)
            distances.append(min_dist_sq)
        total = sum(distances)
        probabilities = [d / total for d in distances]
        next_center = random.choices(data, weights=probabilities, k=1)[0]
        centers.append(next_center)
    return centers



# kmeansObj(P, C)

In [ ]:
# receives in input a set of points P and a set of centers C, and returns the average squared distance of a point of P from its closest center
def kmeansObj(P, C):
    def closest_dist(p):
        return min(squared_distance(p, c) for c in C)
    total_distance = P.map(closest_dist).sum()
    return total_distance / P.count()

In [30]:
import os
import sys
import time
import requests
from pyspark import SparkContext

# Set Spark to use the current Python executable
os.environ["PYSPARK_PYTHON"] = sys.executable

def download_file(url, local_filename):
    print(f"Downloading from {url}...")
    response = requests.get(url)
    response.raise_for_status()  # Raise error if the download fails
    with open(local_filename, 'wb') as f:
        f.write(response.content)
    print(f"File saved to {local_filename}")
    return local_filename

def main(filename, k, k1):
    sc = SparkContext.getOrCreate()

    print(f"Reading data from {filename}...")
    P = readVectorsSeq(filename)  # Your custom function to read the file

    print(f"\nRunning kcenter(P, {k})...")
    start_time = time.time()
    end_time = time.time()
    print(f"Time taken by kcenter: {end_time - start_time:.4f} seconds")

    print(f"\nRunning kmeansPP(P, {k}) and evaluating objective...")
    C_kmeanspp = kmeansPP(P, k)
    obj_kmeanspp = kmeansObj(P, C_kmeanspp)
    print(f"Objective value (kmeans++): {obj_kmeanspp:.4f}")

    print(f"\nRunning kcenter(P, {k1}) to build coreset...")
    X = kcenter(P, k1)
    print(f"Running kmeansPP(X, {k}) on coreset and evaluating objective...")
    C_from_coreset = kmeansPP(sc.parallelize(X), k)
    obj_coreset = kmeansObj(P, C_from_coreset)
    print(f"Objective value (coreset): {obj_coreset:.4f}")

    sc.stop()

if __name__ == "__main__":
    raw_url = "https://raw.githubusercontent.com/TheJojoJoseph/M24DE3041_MLBD_assignment4/858d1d4e905563b50de8d921ba1f315cce2b9b42/Q1-%20UCI%20Spam%20clustering/spambase.data"
    file_path = "/content/spambase.data"
    # downloaded_file = download_file(raw_url, local_file)
    k = 10
    k1 = 50
    main(file_path, k, k1)


Reading data from /content/spambase.data...

Running kcenter(P, 10)...
Time taken by kcenter: 0.0000 seconds

Running kmeansPP(P, 10) and evaluating objective...


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 16) (3d147951e6ea executor driver): java.io.IOException: Cannot run program "D:/AndroidStudio/envs/spark/python.exe": error=2, No such file or directory
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1128)
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1071)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:239)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:139)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at o

# Part 2: Web Search Engine

Part-2: Web-search
40 Marks
1. A function readVectorsSeq(filename) that, given in input the name (or path) filename
of a text file containing the feature points (one point per line with features separated by
comma, as in the file above) transforms it into a list of Vector.
2. A function kcenter(P,k) that receives in input a set of points P and an integer k, and
returns the set C of k centers computed by the Farthest-First Traversal algorithm. You
can represent both P and C as lists of Vector.
3. A function kmeansPP(P,k) that receives in input a set of points P and an integer k, and
returns a set C of k centers computed with the kmeans++ algorithm.
4. A function kmeansObj(P,C) that receives in input a set of points P and a set of centers
C, and returns the average squared distance of a point of P from its closest center (i.e.,
the kmeans objective function for P with centers C, divided by the number of points of P).

1. Runs kcenter(P,k) printing its running time.
2. Runs kmeansPP(P,k) to obtain a set of k centers C, and then runs kmeansOb(P,C)
printing the returned value
3. Runs kcenter(P,k1) to obtain a set of k1 centers X; then runs kmeansPP(X,k) to
extarct a set of k centers C from X, and finally runs kmeansObj(P,C) printing the
returned value. Here the idea is to test whether k1 > k centers extracted with the kcenter
primitive can provide a good coreset on which running kmeans++. Of course, the larger
k1 and the better the set of centers computed by kmeansPP(X,k) .

In this part we will build the basic data structure underlying search engines: an inverted
index. We will use this inverted index to answer some simple search queries.
An inverted index for a set of webpages
Suppose we are given a set of webpages W. For our purposes, each webpage w ∈ W will
be considered to be a sequence of words w1, w2, . . . wk. Another way of representing the
webpage could be to maintain a list of words along with the position(s) of the words in the
webpage. For example consider a web page with the following text:
Data structures is the study of structures for storing data.
This can be represented as:
{(data : 1, 10), (structures : 2, 7), (study : 5), (storing : 9)}
Note that the small connector words like “is”, “the”, “of”, “for” have not been stored. Words
like this are referred to as stop words and are generally removed since they are very
frequent and normally contain no information about the content of the webpage.
This representation of the webpage is similar to the index we see at the back of many books
which tell us the page numbers where certain important terms used in the book may be
found. In fact, we can refer to this as an index for the webpage. In mathematical notation we
would say that given a webpage w = w1, w2, . . . , wk, the index of w is:
{u : i1(u), . . . ,il(u)) : wij(u) = u, 1 ≤ j ≤ l}

An index is used to find the location of a particular string (word) in a specific document or
webpage, but when we move to a collection of webpages, we need to first figure out which of
the web pages contain the string. For this we store an inverted index. Let us try to define an
inverted index formally.
Let us suppose we are given a collection of webpages. For each page p ∈ C, let us denote
by W(p) the set of all words (excluding stop words) that occur in p. Note that

W(C) = ⋃
p∈C
W(p)

is the set of all words in our collection.
An inverted index for C will contain an entry for each word w ∈ W. This entry will contain
tuples of the form (p, k) to indicate that w occurs in the k

th position of page p ∈ C. Using the

notation that p[k] denotes the k

th word of page p, we can say that the inverted index of C is

defined as

Inv(C) = {(w : {(p, k) : p ∈ C, p[k] = w}) : w ∈ W(C)}
For example, consider the following (small) collection of documents.

The inverted index for this would be

The web search problem
The web search problem is defined as follows:
Given a collection of webpages C and a sequence of words q1. . . qk, find the “most relevant”
set of pages p1, p2, . . . pA that contain as many of q1. . . qk as possible and return them in the
order of decreasing “relevance.”
The question of how to measure the relevance of a webpage to a particular query is an
involved question with no easy answers. However, for the purpose of this assignment we will
work with a simple scoring function.
A scoring function for search term relevance
One of the simplest scoring function is term frequency-inverse document frequency. It is
used to measure how important a word w is to a webpage p. It is a product of two factors i.e.
term frequency and inverse document frequency. Given a word w and a webpage p, the
relevance score is defined as

relevancew(p) = tfw(p) × idfw(p)

Term Frequency: It is the total number of occurrence of a word w in a webpage p, denoted by
fw(p). It is normalized by the total number of words in webpage p, denoted by |W(p)|. It is
defined as

tfw(p) =
fw(p)
|W(p)|
1. Data structures is the study of structures for storing data.
2. Structural engineers collect data about structures

{(data : {(1,1), (1,10), (2,4)}),
(structures: {(1,2), (1,7), (2,6)}),
(study : {(1,5)}),
(storing : {(1,9)}),
(structural : {(2,1)}),
(engineers : {(2,2)}),
(collect : {(2,3)}) }

In [52]:
import re

class MySet:
    def __init__(self):
        self.elements = set()

    def addElement(self, element):
        self.elements.add(element)

    def union(self, otherSet):
        result = MySet()
        result.elements = self.elements.union(otherSet.elements)
        return result

    def intersection(self, otherSet):
        result = MySet()
        result.elements = self.elements.intersection(otherSet.elements)
        return result

    def getElements(self):
        return list(self.elements)

    def __contains__(self, item):
        return item in self.elements

    def __iter__(self):
        return iter(self.elements)

    def __len__(self):
        return len(self.elements)

    def __str__(self):
        return str(self.elements)



# class Position

In [49]:

class Position:
    def __init__(self, pageEntry, wordIndex):
        self.pageEntry = pageEntry
        self.wordIndex = wordIndex
    def getPageEntry(self):
        return self.pageEntry
    def getWordIndex(self):
        return self.wordIndex
    def __repr__(self):
        return f"({self.pageEntry.name}, {self.wordIndex})"


In [51]:
class WordEntry:
    def __init__(self, word):
        self.word = word
        self.positions = []

    def addPosition(self, position):
        self.positions.append(position)

    def addPositions(self, positions):
        self.positions.extend(positions)

    def getAllPositions(self):
        return self.positions

    def __repr__(self):
        return f"{self.word}: {self.positions}"


In [48]:
class PageIndex:
    def __init__(self):
        self.word_entries = {}

    def addPositionForWord(self, word, position):
        if word in self.word_entries:
            self.word_entries[word].addPosition(position)
        else:
            entry = WordEntry(word)
            entry.addPosition(position)
            self.word_entries[word] = entry

    def getWordEntry(self, word):
        return self.word_entries.get(word)

    def getAllWordEntries(self):
        return self.word_entries.values()


In [47]:

class PageEntry:
    CONNECTOR_WORDS = {
        "a", "an", "the", "they", "these", "this", "for", "is", "are", "was", "of", "or",
        "and", "does", "will", "whose"
    }
    PUNCTUATION = r"[{}\[\]<>!=()\-.,;:'\"?#]"
    PLURAL_NORMALIZATION = {
        "structures": "structure", "structure": "structure",
        "applications": "application", "application": "application",
        "stacks": "stack", "stack": "stack"
    }
    def __init__(self, pageName):
        self.name = pageName
        self.page_index = PageIndex()
        self.readAndProcessPage(pageName)

    def normalizeWord(self, word):
        word = word.lower()
        word = re.sub(PageEntry.PUNCTUATION, ' ', word)
        word = word.strip()
        if word in PageEntry.PLURAL_NORMALIZATION:
            word = PageEntry.PLURAL_NORMALIZATION[word]
        return word

    def readAndProcessPage(self, pageName):
        try:
            with open("{pageName}", 'r') as file:
                text = file.read()
        except FileNotFoundError:
            print(f"Page '{pageName}' not found!")
            self.name = None
            return

        words = re.split(r'\s+', text)
        index = 1
        for word in words:
            cleaned = self.normalizeWord(word)
            if cleaned and cleaned not in PageEntry.CONNECTOR_WORDS:
                pos = Position(self, index)
                self.page_index.addPositionForWord(cleaned, pos)
            index += 1

    def getPageIndex(self):
        return self.page_index

In [46]:

class MyHashTable:
    def __init__(self):
        self.table = {}
    def addPositionsForWord(self, wordEntry):
        word = wordEntry.word
        if word in self.table:
            self.table[word].addPositions(wordEntry.getAllPositions())
        else:
            self.table[word] = wordEntry
    def getWordEntry(self, word):
        return self.table.get(word)
    def contains(self, word):
        return word in self.table

In [45]:

class InvertedPageIndex:
    def __init__(self):
        self.pages = []
        self.hashtable = MyHashTable()
    def addPage(self, pageEntry):
        self.pages.append(pageEntry)
        for word_entry in pageEntry.getPageIndex().getAllWordEntries():
            self.hashtable.addPositionsForWord(word_entry)
    def getPagesWhichContainWord(self, word):
        word = word.lower()
        entry = self.hashtable.getWordEntry(word)
        result = MySet()
        if entry:
            for pos in entry.getAllPositions():
                result.addElement(pos.getPageEntry())
        return result
    def getAllPositionsForWord(self, word):
        word = word.lower()
        entry = self.hashtable.getWordEntry(word)
        return entry.getAllPositions() if entry else []

# class SearchEngine

In [44]:

class SearchEngine:
    def __init__(self):
        self.index = InvertedPageIndex()

    def performAction(self, actionMessage):
        tokens = actionMessage.strip().split()
        if not tokens:
            return

        action = tokens[0]

        if action == "addPage" and len(tokens) == 2:
            pageName = tokens[1]
            pageEntry = PageEntry(pageName)
            if pageEntry.name is not None:
                self.index.addPage(pageEntry)
                print(f"Page '{pageName}' added.")

        elif action == "queryFindPagesWhichContainWord" and len(tokens) == 2:
            word = tokens[1].lower()
            pages = self.index.getPagesWhichContainWord(word)
            if len(pages) == 0:
                print(f"No webpage contains word {word}")
            else:
                result = sorted(p.name for p in pages)
                print(", ".join(result))

        elif action == "queryFindPositionsOfWordInAPage" and len(tokens) == 3:
            word = tokens[1].lower()
            pageName = tokens[2]
            positions = self.index.getAllPositionsForWord(word)
            found = False
            for pos in positions:
                if pos.getPageEntry().name == pageName:
                    found = True
                    print(f"Word '{word}' found at position(s): {pos.getWordIndex()} in page {pageName}")
            if not found:
                if any(pos.getPageEntry().name == pageName for pos in positions):
                    print(f"Webpage {pageName} does not contain word {word}")
                else:
                    print(f"No webpage {pageName} found.")

        else:
            print(f"Unknown or malformed command: {actionMessage}")

# Main

In [53]:

if __name__ == "__main__":
    engine = SearchEngine()
    actions_file = r"actions.txt"
    with open(actions_file, "r") as f:
        for line in f:
            if line.strip():
                print(f"\n>> {line.strip()}")
                engine.performAction(line)




>> addPage stack_datastructure_wiki
Page 'stack_datastructure_wiki' not found!

>> queryFindPagesWhichContainWord delhi
No webpage contains word delhi

>> queryFindPagesWhichContainWord stack
No webpage contains word stack

>> queryFindPagesWhichContainWord wikipedia
No webpage contains word wikipedia

>> queryFindPositionsOfWordInAPage magazines stack_datastructure_wiki
No webpage stack_datastructure_wiki found.

>> queryFindPagesWhichContainWord allain
No webpage contains word allain

>> addPage stack_cprogramming
Page 'stack_cprogramming' not found!

>> queryFindPagesWhichContainWord allain
No webpage contains word allain

>> queryFindPagesWhichContainWord C
No webpage contains word c

>> queryFindPagesWhichContainWord C++
No webpage contains word c++

>> addPage stack_oracle
Page 'stack_oracle' not found!

>> queryFindPagesWhichContainWord jdk
No webpage contains word jdk

>> addPage stackoverflow
Page 'stackoverflow' not found!

>> queryFindPagesWhichContainWord function
No webpa

# Part 3: PageRank using Spark

Part 3: PageRank on Spark
40 Marks
Get datasets from : https://github.com/dipsankarb/graphs.git
In this assignment you will be implementing PageRank algorithm on Spark. You will be
working with a small random graph that is provided. The graph has n=1000 nodes, and
described later.

addPage x : Add webpage x to the search engine database. The contents of the
webpage are stored in a file named x in the webpages folder.
queryFindPagesWhichContainWord x : Print the name of the webpages which contain
the word x. The list of webpage names should be comma separated. If the word is not
found in any webpage, then print “No webpage contains word x”
queryFindPositionsOfWordInAPage x y : Print the word indices where the word x is
found in the document y. The word indices should be separated by a comma. If the word
x is not found in webpage y, then print “Webpage y does not contain word x”. If the
webpage is not added in database, then print “No webpage y found”.

Convert each word to lowercase.
Do not store the connector words in the search engine. However, consider them when
you calculate the word indices. Here is a list of connector words: { a, an, the, they, these,
this, for, is, are, was, of, or, and, does, will, whose }.
Replace the punctuation marks with a space. Here is a list of punctuations: { } [ ] < > = ( )
. , ; ’ ” ? # ! - :
Plural and singular form: Assume that these words are same: (stack, stacks), (structure,
structures), (application, applications).
We have given you a set of connector words, punctuation marks, and singular-plural
entries. Consider these list of words as exhaustive. You do not need to make any more
exceptions in your code.
actions.txt contains some simple search queries.
answers.txt contains the answers to actions.txt for you to check your code.

m=8192 edges. 1000 of these edges form a directed cycle which ensures that the graph is
connected. It is trivial to see that the existence of such a cycle ensures that there are no
dead ends in the graph. The dataset may have multiple directed edges between a pair of
nodes which should be treated as a single edge. The first column in the provided dataset
whole.txt is the source vertex and the second column denotes the destination.
Assume a directed graph G = (V , E) has n nodes and m edges and all nodes having
positive out-degrees, then M is a matrix which is an n × n matrix as defined such that for any
(i, j) ∈ [1, n]:

Mi,j = {

Here, deg(i) is out-degree of the node i. If there are multiple outgoing edges, then treat them
as a single node. By definition of PageRank, assuming 1 − β as the teleport probability, and
denoting the PageRank vector by the column vector r, we have the following relation:

r =
1 − β
n
A + βMr

Here, A is the n × 1 unit vector. Based on these equations, the iterative PageRank works as
follows:

Run this experiment on Spark for 40 iterations (assuming β = 0.8) and obtain the PageRank
vector r. In particular, process the matrix M as an RDD and then report the following:

Before running the experiment on the whole graph on the server, run the code on a smaller
part of the graph given in small.txt with 53 nodes. The top most score in this small graph is
0.036

Small.txt dataset

In [54]:
from pyspark import SparkContext

def load_graph(file_path):
    sc = SparkContext.getOrCreate()
    lines = sc.textFile(file_path)
    edges = lines.map(lambda line: tuple(map(int, line.strip().split()))).distinct()
    links = edges.groupByKey().mapValues(list).cache()
    return sc, links

def initialize_ranks(links):
    n = links.count()
    ranks = links.mapValues(lambda _: 1.0 / n)
    return ranks, n

def run_pagerank(links, ranks, n, beta=0.8, iterations=40):
    for _ in range(iterations):
        contribs = links.join(ranks).flatMap(
            lambda x: [(dest, x[1][1] / len(x[1][0])) for dest in x[1][0]]
        )
        ranks = contribs.reduceByKey(lambda a, b: a + b) \
                       .mapValues(lambda rank: (1 - beta) / n + beta * rank)
    return ranks

def main():
    file_path = "small.txt"
    sc, links = load_graph(file_path)
    ranks, n = initialize_ranks(links)
    final_ranks = run_pagerank(links, ranks, n)

    top5 = final_ranks.takeOrdered(5, key=lambda x: -x[1])
    bottom5 = final_ranks.takeOrdered(5, key=lambda x: x[1])
    print("-----------Top 5 high PageRank:----")
    for node, score in top5:
        print(f"Node {node}: {score:.6f}")
    print("------- Bottom 5 lowest PageRank ----")
    for node, score in bottom5:
        print(f"Node {node}: {score:.6f}")
    sc.stop()

if __name__ == "__main__":
    main()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 18) (3d147951e6ea executor driver): java.io.IOException: Cannot run program "D:/AndroidStudio/envs/spark/python.exe": error=2, No such file or directory
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1128)
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1071)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:239)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:139)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at o

Whole.txt dataset

In [ ]:

def initialize_ranks(links):
    n = links.count()
    ranks = links.mapValues(lambda _: 1.0 / n)
    return ranks, n

def run_pagerank(links, ranks, n, beta=0.8, iterations=40):
    for _ in range(iterations):
        contribs = links.join(ranks).flatMap(
            lambda x: [(dest, x[1][1] / len(x[1][0])) for dest in x[1][0]]
        )
        ranks = contribs.reduceByKey(lambda a, b: a + b) \
                       .mapValues(lambda rank: (1 - beta) / n + beta * rank)
    return ranks
def main():
    file_path = "whole.txt"
    sc, links = load_graph(file_path)
    ranks, n = initialize_ranks(links)
    final_ranks = run_pagerank(links, ranks, n)

    top5 = final_ranks.takeOrdered(5, key=lambda x: -x[1])
    bottom5 = final_ranks.takeOrdered(5, key=lambda x: x[1])
    print("---------Top 5 nodes with highest PageRank----")
    for node, score in top5:
        print(f"Node {node}: {score:.6f}")
    print("------ Bottom 5 nodes with lowest PageRank ----------")
    for node, score in bottom5:
        print(f"Node {node}: {score:.6f}")
    sc.stop()
if __name__ == "__main__":
    main()


Top 5 nodes with highest PageRank:
Node 263: 0.002020
Node 537: 0.001943
Node 965: 0.001925
Node 243: 0.001853
Node 285: 0.001827

Bottom 5 nodes with lowest PageRank:
Node 558: 0.000329
Node 93: 0.000351
Node 62: 0.000353
Node 424: 0.000355
Node 408: 0.000388
